# BlueBook for Bulldozers

### Introduction

In this lesson, we'll use the [bluebook for bulldozers](https://www.kaggle.com/c/bluebook-for-bulldozers/data) dataset.  This was a Kaggle competition so we don't need to determine the target variable.  It's given for us.  

Let's load up our data, and work through our data.

### Loading our Data

In [1]:
import pandas as pd 

df = pd.read_csv('./TrainAndValid.csv')

/Users/jeff/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
df[:10000].to_csv('./bulldozers.csv')

In [3]:
df.shape

(412698, 53)

In [7]:
# df.columns

We have a total of 53 columns in our dataset.  Our goal is to predict the SalePrice.  Let's get started.

### Reducing our problem

We can start by separating our target variable from our dataset.

In [5]:
X = df.drop(columns = 'SalePrice')

In [6]:
y = df['SalePrice']

Let's start by seeing if we have any non-unique values in the dataset.

In [8]:
def find_all_same(df):
    return [col for col in df.columns if len(df[col].unique()) == 1]

In [9]:
find_all_same(X)

[]

### Datetime Values

So we see that all of our columns in the dataset are different.  Next let's see if we have any columns date columns we can coerce.

In [17]:
obj_df = df.select_dtypes(include = 'object')

In [13]:
obj.shape

(412698, 45)

In [14]:
def contains_date(column):
#     remove nas first, potentially use all
    regex_string = (r'^\d{1,2}-\d{1,2}-\d{4}$|^\d{4}-\d{1,2}-\d{1,2}$' + 
'|^\d{1,2}\/\d{1,2}\/\d{4}$|^\d{4}\/\d{1,2}\/\d{1,2}$')
    return column.str.contains(regex_string).any()

In [24]:
# potential_date_cols = [col for col in obj_df.columns if contains_date(obj_df[col])]

In [25]:
# potential_date_cols

Our contains date column apparently isn't working, but that doesn't mean there is no date.

In [27]:
# obj_df[:2].T

### Numeric Columns

In [29]:
def contains_numbers(column):
    # matches price or percentage     
    regex_string = (r'^(?!.*www|.*-|.*\/|.*[A-Za-z]|.* ).*\d.*')
#     regex_string = (r'\$\d+.*|\d+.*\%$|^\d+.*$')
    return column.str.contains(regex_string).any()

In [31]:
number_cols = obj_df.apply(lambda col: contains_numbers(col))

In [35]:
number_cols[number_cols == True]

# fiModelDesc          True
# fiBaseModel          True
# fiSecondaryDesc      True
# fiModelSeries        True
# fiModelDescriptor    True
# Blade_Width          True
# Tire_Size            True
# dtype: bool

fiModelDesc          True
fiBaseModel          True
fiSecondaryDesc      True
fiModelSeries        True
fiModelDescriptor    True
Blade_Width          True
Tire_Size            True
dtype: bool

So we can find seven columns that contain numbers.  Let's start by looking at these seven columns.

Select the columns from the name of the series above.

In [37]:
number_cols[number_cols == True].index

Index(['fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'Blade_Width', 'Tire_Size'],
      dtype='object')

And then select the columns from the dataframe.

In [40]:
obj_df[number_cols[number_cols == True].index]['Tire_Size'].value_counts()

None or Unspecified    47823
20.5                   15773
14"                     9111
23.5                    8760
26.5                    4635
17.5                    3971
29.5                    2767
17.5"                   1815
13"                      776
20.5"                    737
15.5                     610
15.5"                    463
23.5"                    309
7.0"                      56
23.1"                     20
10"                        9
10 inch                    3
Name: Tire_Size, dtype: int64